# Capstone Project - The Battle of Neighborhoods

## Introduction - Business Problem

In this project, we will find an location(neighborhood) with the most cafes. Specifically, this report will be targeted to stakeholders interested in opening an cafe in **Toronto, ON, Canada**.

There are lots of cafes in Toronto, so we will try to detect locations that are not already crowded with cafes. We will present **the best 5 places** so that best possible final location can be chosen by stakeholders.

## Data

centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**

the information of neighborhoods and their population of Toronto is from Scraping Wikipedia 

number of cafes and their location in every neighborhood will be obtained using **Foursquare API**.

### Candidates

Let's prepare for solving the business problem by data analysis

In [4]:
import numpy as np
import pandas as pd
import json
from bs4 import BeautifulSoup
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim

Let's get information about borough and neighborhoods in Canada!

The dataset is extracted from Wikipedia using BeautifulSoup.

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, 'html5lib')

In [106]:
soup.find('table')

<table cellpadding="2" cellspacing="0" rules="all" style="width:100%; border-collapse:collapse; border:1px solid #ccc;">

<tbody><tr>
<td style="width:11%; vertical-align:top; color:#ccc;">
<p><b>M1A</b><br/><span style="font-size:85%;"><i>Not assigned</i></span>
</p>
</td>
<td style="width:11%; vertical-align:top; color:#ccc;">
<p><b>M2A</b><br/><span style="font-size:85%;"><i>Not assigned</i></span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M3A</b><br/><span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>)</span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M4A</b><br/><span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>)</span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M5A</b><br/><span style="font-size:85%;"><a h

In [6]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)


df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


And merge with latitude and longtitude data

In [7]:
df_geo = pd.read_csv('Geospatial_Coordinates.csv')
df_geo.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df = pd.merge(df, df_geo, how='inner', on='PostalCode')
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


Let's find the latitude & longitude of Berlin city center using Google Maps geocoding API.

In [8]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Show on Map

In [9]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.4,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Get the venues that are in each neighborhood within 300 meters.

In [13]:
CLIENT_ID = 'UGAZR2WF3DRKOBUQ2C5VMOSYVOQN0HWBSY2ZKUOSTRTLYICJ'
CLIENT_SECRET = '1YNGALR2KWANKSYQDTUHI4QB0DVX4CB0LPD2ED2UCHKS4SZZ'
VERSION=20180604

In [57]:
def FindVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, 300)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results]
        )
        
    find_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    find_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(find_venues)

In [58]:
toronto_venues = FindVenues(df['Neighborhood'], df['Latitude'], df['Longitude'])
toronto_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Brookbanks Pool,43.751389,-79.332184,Pool
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
5,Victoria Village,43.725882,-79.315572,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
6,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place
7,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
8,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
9,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center


In [61]:
toronto_venues.shape

(842, 7)

In [60]:
len(toronto_venues['Venue Category'].unique())

192

### Pre-processing

We can say 'Café' and 'Coffee shop' are the same, so Let's comebine together to 'Coffe Shop'

In [62]:
for i in toronto_venues.index:
    if toronto_venues.loc[i, 'Venue Category'] == 'Café':
        toronto_venues.loc[i, 'Venue Category'] = 'Coffee Shop'

In [63]:
toronto_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Brookbanks Pool,43.751389,-79.332184,Pool
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
5,Victoria Village,43.725882,-79.315572,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
6,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place
7,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
8,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
9,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center


## Analysis

### The most common venues

Let's see the most 5 common venues' categories for each neighborhood

In [64]:
toronto_categories = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_categories['Neighborhood'] = toronto_venues['Neighborhood']

In [65]:
a = list(toronto_categories)
a.index('Neighborhood')

136

In [66]:
fixcolumns = [toronto_categories.columns[163]] + list(toronto_categories.columns[:163]) + list(toronto_categories.columns[164:])
toronto_categories = toronto_categories[fixcolumns]
toronto_categories.head()

,Spa,Accessories Store,Adult Boutique,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arepa Restaurant,Art Gallery,...,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
toronto_grouped = toronto_categories.groupby('Neighborhood').sum().reset_index()
toronto_grouped.head()

,Neighborhood,Spa,Accessories Store,Adult Boutique,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arepa Restaurant,...,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Alderwood, Long Branch",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Bathurst Manor, Wilson Heights, Downsview North",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Bedford Park, Lawrence Manor East",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Berczy Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant   1.0
1             Breakfast Spot   1.0
2                  Pet Store   0.0
3               Liquor Store   0.0
4                     Lounge   0.0


----Alderwood, Long Branch----
         venue  freq
0     Pharmacy   1.0
1          Gym   1.0
2  Pizza Place   1.0
3  Coffee Shop   1.0
4          Pub   1.0


----Bathurst Manor, Wilson Heights, Downsview North----
                 venue  freq
0          Coffee Shop   2.0
1          Supermarket   1.0
2    Mobile Phone Shop   1.0
3  Fried Chicken Joint   1.0
4           Restaurant   1.0


----Bedford Park, Lawrence Manor East----
                     venue  freq
0              Coffee Shop   2.0
1           Sandwich Place   2.0
2       Italian Restaurant   2.0
3  Comfort Food Restaurant   1.0
4          Thai Restaurant   1.0


----Berczy Park----
                venue  freq
0        Liquor Store   1.0
1                 Pub   1.0
2  Belgian Restaurant   1.0
3    

                 venue  freq
0  Sporting Goods Shop   3.0
1                 Bank   2.0
2           Sports Bar   1.0
3         Burger Joint   1.0
4     Sushi Restaurant   1.0


----Little Portugal, Trinity----
                   venue  freq
0                    Bar   4.0
1  Vietnamese Restaurant   2.0
2       Asian Restaurant   2.0
3            Men's Store   1.0
4    Japanese Restaurant   1.0


----Milliken, Agincourt North, Steeles East, L'Amoreaux East----
                venue  freq
0                Park   1.0
1        Intersection   1.0
2                Lake   0.0
3  Light Rail Station   0.0
4        Liquor Store   0.0


----Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West----
                  venue  freq
0  Fast Food Restaurant   1.0
1        Hardware Store   1.0
2           Wings Joint   1.0
3     Convenience Store   1.0
4       Supplement Shop   1.0


----Moore Park, Summerhill East----
                        venue  freq
0          

4                Liquor Store   0.0


----York Mills West----
                 venue  freq
0                 Park   1.0
1    Convenience Store   1.0
2  Monument / Landmark   0.0
3         Liquor Store   0.0
4               Lounge   0.0




In [69]:
def return_most_common_venues(row):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:5]

In [70]:
columns=['Neighborhood', '1st Most Common Venue', '2nd Most Common Venue', '3rd Most Common Venue', '4th Most Common Venue', 
        '5th Most Common Venue']    
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :])

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Latin American Restaurant,Breakfast Spot,Yoga Studio,Colombian Restaurant,Dumpling Restaurant
1,"Alderwood, Long Branch",Coffee Shop,Gym,Pharmacy,Pizza Place,Pub
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Shopping Mall,Deli / Bodega,Sushi Restaurant,Supermarket
3,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Sandwich Place,Juice Bar,Restaurant
4,Berczy Park,Pub,Concert Hall,Breakfast Spot,Belgian Restaurant,Beer Bar
5,"Birch Cliff, Cliffside West",Coffee Shop,Construction & Landscaping,Pizza Place,Brewery,Yoga Studio
6,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Sandwich Place,Sporting Goods Shop,Pizza Place,Breakfast Spot
7,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,Coffee Shop
8,Caledonia-Fairbanks,Park,Women's Store,Accessories Store,Design Studio,Electronics Store
9,Cedarbrae,Athletics & Sports,Hakka Restaurant,Bank,Caribbean Restaurant,Thai Restaurant


Among the Neighborhood, Let's handle the neighborhoods that there is no Coffee Shop in top 5 the Most Common Venue 

In [71]:
no_Coffee_Shop = pd.DataFrame(columns=columns)
for i in neighborhoods_venues_sorted.index:
    if 'Coffee Shop' not in list(neighborhoods_venues_sorted.loc[i]):
        no_Coffee_Shop.loc[i] = neighborhoods_venues_sorted.loc[i]
        
no_Coffee_Shop = no_Coffee_Shop.reset_index()
no_Coffee_Shop = no_Coffee_Shop.drop(columns='index')
no_Coffee_Shop.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Latin American Restaurant,Breakfast Spot,Yoga Studio,Colombian Restaurant,Dumpling Restaurant
1,Berczy Park,Pub,Concert Hall,Breakfast Spot,Belgian Restaurant,Beer Bar
2,Caledonia-Fairbanks,Park,Women's Store,Accessories Store,Design Studio,Electronics Store
3,Cedarbrae,Athletics & Sports,Hakka Restaurant,Bank,Caribbean Restaurant,Thai Restaurant
4,Church and Wellesley,Bubble Tea Shop,Japanese Restaurant,Pizza Place,Ice Cream Shop,Diner
5,"Clarks Corners, Tam O'Shanter, Sullivan",Shopping Mall,Bank,Yoga Studio,Escape Room,Dumpling Restaurant
6,"Cliffside, Cliffcrest, Scarborough Village West",Skating Rink,Motel,American Restaurant,Yoga Studio,Design Studio
7,Davisville North,Gym / Fitness Center,Breakfast Spot,Convenience Store,Pool,Yoga Studio
8,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",Bar,Sandwich Place,Turkish Restaurant,Discount Store,Yoga Studio
9,Don Mills North,Tennis Court,Yoga Studio,Escape Room,Dumpling Restaurant,Donut Shop


In [72]:
no_Coffee_Shop.shape

(48, 6)

### Population of each neighborhood

We need only the most recent population, so extract population in 2016.

In [122]:
neighborhood_pop_raw = pd.read_csv('neighbourhood-profiles-2016-csv.csv')
neighborhood_pop_raw.head()

,_id,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,...,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,1,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,...,37,7,137,64,60,94,100,97,27,31
1,2,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,...,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,Emerging Neighbourhood
2,3,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2016","2,731,571","29,113","23,757","12,054","30,526",...,"16,936","22,156","53,485","12,541","7,865","14,349","11,817","12,528","27,593","14,804"
3,4,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2011","2,615,060","30,279","21,988","11,904","29,177",...,"15,004","21,343","53,350","11,703","7,826","13,986","10,578","11,652","27,713","14,687"
4,5,Population,Population and dwellings,Census Profile 98-316-X2016001,Population Change 2011-2016,4.50%,-3.90%,8.00%,1.30%,4.60%,...,12.90%,3.80%,0.30%,7.20%,0.50%,2.60%,11.70%,7.50%,-0.40%,0.80%


In [140]:
neighborhood_pop = neighborhood_pop_raw[neighborhood_pop_raw['Characteristic']=='Population, 2016']
neighborhood_pop

,_id,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,...,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
2,3,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2016","2,731,571","29,113","23,757","12,054","30,526",...,"16,936","22,156","53,485","12,541","7,865","14,349","11,817","12,528","27,593","14,804"


In [142]:
d = neighborhood_pop.columns[:5]
neighborhood_pop = neighborhood_pop.transpose().drop(d).reset_index()
neighborhood_pop

,index,2
0,City of Toronto,"2,731,571"
1,Agincourt North,"29,113"
2,Agincourt South-Malvern West,"23,757"
3,Alderwood,"12,054"
4,Annex,"30,526"
...,...,...
136,Wychwood,"14,349"
137,Yonge-Eglinton,"11,817"
138,Yonge-St.Clair,"12,528"
139,York University Heights,"27,593"


In [143]:
neighborhood_pop.columns=['Neighborhood','Population']
neighborhood_pop

,Neighborhood,Population
0,City of Toronto,"2,731,571"
1,Agincourt North,"29,113"
2,Agincourt South-Malvern West,"23,757"
3,Alderwood,"12,054"
4,Annex,"30,526"
...,...,...
136,Wychwood,"14,349"
137,Yonge-Eglinton,"11,817"
138,Yonge-St.Clair,"12,528"
139,York University Heights,"27,593"


### Result: The best 5 lacation

Put the population information to dataframe 'no_Coffee_Shop'

And sort by population, ascending False.

In [160]:
result = pd.merge(no_Coffee_Shop, neighborhood_pop, how='inner', on='Neighborhood')
result = result.sort_values('Population', ascending=False).head(5).reset_index()
result = result.drop(columns='index')
result

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Population
0,Woburn,Korean BBQ Restaurant,Design Studio,Electronics Store,Dumpling Restaurant,Donut Shop,"53,485"
1,The Beaches,Park,Trail,Department Store,Dumpling Restaurant,Donut Shop,"21,567"
2,Weston,Park,Jewelry Store,Escape Room,Dumpling Restaurant,Donut Shop,"17,992"
3,Hillcrest Village,Golf Course,Athletics & Sports,Dog Run,Fast Food Restaurant,Mediterranean Restaurant,"16,934"
4,Scarborough Village,Playground,Yoga Studio,Department Store,Dumpling Restaurant,Donut Shop,"16,724"


In [162]:
best5 = result['Neighborhood']

## Conclusion

The best 5 locations to start up cafe business are....

In [163]:
best5

0                 Woburn
1            The Beaches
2                 Weston
3      Hillcrest Village
4    Scarborough Village
Name: Neighborhood, dtype: object